In [14]:
import re

import pickle
from sklearn.neighbors import KNeighborsClassifier

In [15]:
with open('misconduct_train.pkl', 'rb') as train_file:
    misconduct_train = pickle.load(train_file)

In [11]:
dummyized_cols = [
    'misconduct_type',
    'court_type',
    'enforcement_agency',
    'contracting_party'
]

indicator_features = [
    col for col in misconduct_train.columns
    if re.search(f'^({"|".join(dummyized_cols)})_', col)
]

In [12]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(
    misconduct_train.loc[:, indicator_features],
    misconduct_train['disposition_type']
)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [ ]:
with open('misconduct_test.pkl', 'rb') as test_file:
    misconduct_test = pickle.load(test_file)

misconduct_test[99]